In [111]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key


In [112]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/challenge.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain Inches,Snow Inches
0,0,Sfantu Gheorghe,RO,2020-05-12 03:54:28,45.87,25.78,55.81,85,87,5.39,overcast clouds,0.00,0.0
1,1,Conde,BR,2020-05-12 03:54:29,-7.26,-34.91,75.20,100,20,5.82,few clouds,0.00,0.0
2,2,Butaritari,KI,2020-05-12 03:50:08,3.07,172.79,84.06,76,57,11.97,broken clouds,0.00,0.0
3,3,Atuona,PF,2020-05-12 03:54:29,-9.80,-139.03,80.19,74,53,17.90,broken clouds,0.00,0.0
4,4,Harper,LR,2020-05-12 03:51:25,4.38,-7.72,78.37,88,100,9.26,light rain,0.15,0.0


In [113]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 40
What is the maximum temperature you would like for your trip? 60


In [114]:
#Ask the customer if they want it to be raining or snowing.
rainfall = (input("Do you want it to be raining? (yes/no) "))
snowfall = (input("Do you w60ant it to be snowing? (yes/no)"))

Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no)yes


In [115]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

if rainfall == "yes":
    preferred_rain_df = preferred_cities_df.loc[(preferred_cities_df["Rain Inches"] > 0)]
else:
    preferred_rain_df = preferred_cities_df.loc[(preferred_cities_df["Rain Inches"] == 0.0)]

if snowfall == "yes":
    preferred_snow_df = preferred_cities_df.loc[(preferred_cities_df["Snow Inches"] > 0)]
else:
    preferred_snow_df = preferred_cities_df.loc[(preferred_cities_df["Snow Inches"] == 0.0)]

preferred_cities_df.head(10)


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain Inches,Snow Inches
0,0,Sfantu Gheorghe,RO,2020-05-12 03:54:28,45.87,25.78,55.81,85,87,5.39,overcast clouds,0.00,0.0
5,5,Lazi,UA,2020-05-12 03:54:29,48.07,24.23,53.38,84,52,8.39,light rain,0.15,0.0
6,6,Coihaique,CL,2020-05-12 03:54:29,-45.58,-72.07,46.40,81,75,6.93,light rain,0.00,0.0
7,7,Dunedin,NZ,2020-05-12 03:53:40,-45.87,170.50,57.99,66,100,1.95,overcast clouds,0.00,0.0
8,8,Tuatapere,NZ,2020-05-12 03:54:29,-46.13,167.68,53.78,69,6,3.24,clear sky,0.00,0.0
12,12,Frankfort,ZA,2020-05-12 03:54:30,-27.27,28.48,46.24,54,0,4.09,clear sky,0.00,0.0
15,15,Mataura,NZ,2020-05-12 03:50:08,-46.19,168.86,52.38,71,100,1.59,overcast clouds,0.00,0.0
16,16,Valladolid,ES,2020-05-12 03:54:30,41.58,-4.67,52.00,93,75,2.24,broken clouds,0.00,0.0
18,18,Lebu,CL,2020-05-12 03:54:31,-37.62,-73.65,51.39,85,71,11.12,broken clouds,0.00,0.0
20,20,Pacifica,US,2020-05-12 03:54:31,37.61,-122.49,59.00,100,90,14.99,mist,0.25,0.0


In [116]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description","Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Sfantu Gheorghe,RO,55.81,overcast clouds,45.87,25.78,
5,Lazi,UA,53.38,light rain,48.07,24.23,
6,Coihaique,CL,46.40,light rain,-45.58,-72.07,
7,Dunedin,NZ,57.99,overcast clouds,-45.87,170.50,
8,Tuatapere,NZ,53.78,clear sky,-46.13,167.68,
12,Frankfort,ZA,46.24,clear sky,-27.27,28.48,
15,Mataura,NZ,52.38,overcast clouds,-46.19,168.86,
16,Valladolid,ES,52.00,broken clouds,41.58,-4.67,
18,Lebu,CL,51.39,broken clouds,-37.62,-73.65,
20,Pacifica,US,59.00,mist,37.61,-122.49,


In [117]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [118]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df.head(10)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Sfantu Gheorghe,RO,55.81,overcast clouds,45.87,25.78,Hotel Park Sfantu Gheorghe
5,Lazi,UA,53.38,light rain,48.07,24.23,Olenka
6,Coihaique,CL,46.40,light rain,-45.58,-72.07,Diego De Almagro Coyhaique
7,Dunedin,NZ,57.99,overcast clouds,-45.87,170.50,Scenic Hotel Southern Cross
8,Tuatapere,NZ,53.78,clear sky,-46.13,167.68,Tom Cruse Refreshment Company
12,Frankfort,ZA,46.24,clear sky,-27.27,28.48,Formule 11 Accomodation
15,Mataura,NZ,52.38,overcast clouds,-46.19,168.86,Ellie's Villa
16,Valladolid,ES,52.00,broken clouds,41.58,-4.67,Hotel Ruta del Duero
18,Lebu,CL,51.39,broken clouds,-37.62,-73.65,Hostal Las Lilas
20,Pacifica,US,59.00,mist,37.61,-122.49,"Pacifica Lighthouse, Trademark Collection by W..."


In [119]:
hotel_df.count()

City                   157
Country                154
Max Temp               157
Current Description    157
Lat                    157
Lng                    157
Hotel Name             157
dtype: int64

In [120]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [121]:
# Add a marker layer map of temperature and current description for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [122]:
# Create the output file (CSV).
output_data_file = "Weather_data/WeatherPy_vacation.csv"
# Export the hotel_df into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")